# Multi-Car Racing PPO - GitHub Workflow

This notebook acts as a tailored environment to run the code directly from your [GitHub Repository](https://github.com/yuvimalik/Racing_Gym_RL).

## Setup Instructions
1. **Runtime**: Ensure you are using a GPU Runtime (Runtime -> Change runtime type -> GPU).
2. **Clone & Install**: Run the setup cells below to clone your code and install dependencies.
3. **Train or Evaluate**: Use the provided cells to run your `train.py` or `evaluate.py` scripts.

In [ ]:
# 1. Clone the Repository
!git clone https://github.com/yuvimalik/Racing_Gym_RL.git
%cd Racing_Gym_RL
!git pull origin main  # Ensure we have the latest version

In [ ]:
# 2. Install Dependencies
# System dependencies for rendering
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Python dependencies
!pip install -r requirements.txt
# Install the multi_car_racing environment (no dependencies to avoid conflicts)
!pip install -q git+https://github.com/igilitschenski/multi_car_racing.git --no-deps

print("Dependencies installed.")

## Option A: Train a New Model
Run the training script using the configuration file in the repo.

In [ ]:
# Run Training
!python train.py --config config/multi_car_config.yaml

## Option B: Evaluate a Model
1. **Upload**: If you have a trained model `.zip` on your computer, upload it to the `Racing_Gym_RL/models/` folder (use the file browser on the left).
3. **Run Evaluation**: Execute the script below.

In [ ]:
# Creates models directory if it doesn't exist (for uploading)
!mkdir -p models

In [ ]:
# EXAMPLE: Run evaluation on a specific model
# Replace 'models/final_model.zip' with your uploaded model name if different
!python evaluate.py --model models/final_model.zip --episodes 3 --no-video

## Option C: Helper to Display Video
The `evaluate.py` script saves videos to `results/`. Use this cell to pick the latest video and play it in the notebook.

In [ ]:
import glob
import os
from IPython.display import Video, display

# Find the latest video file in results/
list_of_files = glob.glob('results/*.mp4') 
if list_of_files:
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"Displaying: {latest_file}")
    display(Video(latest_file, embed=True, html_attributes='controls loop autoplay'))
else:
    print("No video files found in results/.")